This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
from collections import namedtuple

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-06-08&end_date=2017-06-08&api_key=" + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-06-08',
  'end_date': '2017-06-08',
  'data': [['2017-06-08',
    47.8,
    47.8,
    46.27,
    46.27,
    None,
    1945.0,
    90599.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)

In [5]:
r = requests.get("https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY)

#### 2. Convert the returned JSON object into a Python dictionary

In [6]:
r_dict = r.json()

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period. 

In [7]:
# Create list of data from JSON dictionary
data_list = r_dict['dataset']['data']

# Create namedtuple object to hold date and daily prices
DailyPrice = namedtuple('DailyPrice', 'date prices')

In [8]:
# Store daily prices for each date in list

total_list = []

for num in range(len(data_list)):
    
    temp_list = []
    day = DailyPrice(date=data_list[num][0], prices=data_list[num][1:7])
    
    temp_list.append(day.date)
    temp_list.append(day.prices)
    
    total_list.append(temp_list)

In [9]:
# Reverse list in place to get dates in chronological order
total_list.reverse()

In [10]:
# Calculate maximum and minimum opening prices and associated dates

max_open = 0
min_open = 1000
min_date = ''
max_date = ''

for num in range(len(total_list)):
    
    date = total_list[num][0]
    open_price = total_list[num][1][0]
    
    try:
        if open_price > max_open:
            
            max_open = open_price
            max_date = date
            
        if open_price < min_open:
            
            min_open = open_price
            min_date = date
            
    except:               
        pass
    
print(f'The highest opening price was {max_open} on {max_date}')
print(f'The lowest opening price was {min_open} on {min_date}')

The highest opening price was 53.11 on 2017-12-14
The lowest opening price was 34.0 on 2017-01-24


#### 4. What was the largest change in any one day (based on High and Low price)?

In [11]:
# Calculate the maximum difference between maximum and minimum price for each day

max_diff = 0
max_diff_date = ''

for num in range(len(total_list)):
    
    diff = total_list[num][1][1] - total_list[num][1][2]
    
    if diff > max_diff:    
        
        max_diff = diff
        max_diff_date = total_list[num][0]

print(f'The largest price change in any one day was {max_diff} on {max_diff_date}')

The largest price change in any one day was 2.8100000000000023 on 2017-05-11


#### 5. What was the largest change between any two days (based on Closing Price)?

In [12]:
# Calculate the maximum difference between any two days' closing prices

max_close = 0
min_close = 1000
max_close_date = ''
min_close_date = ''

for num in range(1, len(total_list)):
    
    date = total_list[num][0]
    close_price = total_list[num][1][3]
    
    if close_price > max_close:  
        
        max_close = close_price
        max_close_date = date
        
    if close_price < min_close:    
        
        min_close = close_price
        min_close_date = date   
        
diff = max_close - min_close
                               
print(f'The largest change between any two days was {diff}. \nMinimum Close Date: {min_close_date} \nMaximum Close Date: {max_close_date}')

The largest change between any two days was 19.03. 
Minimum Close Date: 2017-01-23 
Maximum Close Date: 2017-12-13


#### 6. What was the average daily trading volume during this year?

In [13]:
# Calculate average trading volume for the year

trading_vol_list = []

for num in range(len(total_list)):
    
    trading_vol_list.append(total_list[num][1][5])
    
avg_trade_vol = sum(trading_vol_list) / len(trading_vol_list)

print(f'The average daily trading volume during the year was {avg_trade_vol}')

The average daily trading volume during the year was 89124.33725490196
